# This notebook

- Overview of testing tools focused at the unit level 
- Some depth, but most of these topics could be their own talk
- Disclaimer this talk is for exposure, not for setting team standards

### Tools:
- Pycodestyle
- Pyflakes
- Pylint
- Doctest
- Mypy
- Unittest
- Pytest 
- Hypothesis
- Stubs and Mocks
- Coverage
- Tox

# Why should you use tests?

### When test are well written they allow for the following:
- More confidence in committed code
- Easier refactoring of code as you can test things quickly after making changes
- Extra information on how code is expected to run
- Foundation for other good things like good CI/CD 

### Drawbacks:
- Testing is not a panacea for code bugs
- Badly written tests can be as detrimental as good test are helpful
- Even having too many "good" tests can confuse team members and slow development
- Like most things, testing a balancing act

# Linters
- A few common tools you likely have already use... 

### Pycodestyle
- https://pycodestyle.pycqa.org/en/latest/
- The static analysis tool formally known as pep8
- Tells you how non pep8 your code is

### Pyflakes
- https://pypi.org/project/pyflakes/
- Not focused on pep8 or style, just potential bugs
- More conservative than other tools like pylint for calling out items 
- Lighter weight than some other tools like because only compares a single file at a time

### Pylint
- http://pylint.pycqa.org/en/latest/
- Checks style and code content 
- Can be a jerk.. "Your code on a scale from 0 to 10 is -12.7, please try again" -pylint
- Can have a fair amount of false positives and other noise

# Other static checks

### Doctest
- https://docs.python.org/3.8/library/doctest.html
- Part of the standard library
- Python 2 and 3 compatible 
- Not really for rigorous testing, more of an example with added benefit of some test checks
- "Executable documentation"

In [ ]:
import src.examples.coding as demo

In [ ]:
help(demo.division)

In [ ]:
demo.division??

- Run from cli with doctest:

In [ ]:
%%bash
python -m doctest -v src/examples/coding.py 

- Or by adding this in your module... 

In [ ]:
if __name__ == "__main__": 
    import doctest 
    doctest.testmod()

### Mypy
- https://mypy.readthedocs.io/en/stable/
- Typing info... https://docs.python.org/3/library/typing.html
- static type hinting, still lets you use wrong type in run time
- Several variants out there from various companies 
- Old and new way to use based on python versions

##### How some people feel about mypy in python...

In [ ]:
from IPython.display import Image
Image(filename='mypy.png') 

- Older method using comments
- Python 2 & 3

In [ ]:
#from typing import List
def division(num, denom): # type: (int, int) -> float
    temp = ["temp variable"] # type: List[str]
    return num / denom 

- Newer method 3.5(?)+ doesn't need to use comments
- 3.6(?) allows for single variable type checks
- 3.9 will apparently remove need for typing...?

- Some common usable types...

In [ ]:
Optional[int]=None
Seqeuence[int]
Sequence[optional]

# Unittest Frameworks

### Test Suggestions
- Results should be repeatable
- Tests should target one thing !!
- Tests should be independent !! 
- Try and test boundary conditions, potential errors, normal working, and None type considerations
- Tests should run fast (If they're slow you won't run them as often...)
- Use overly descriptive names as much possible for better error analysis on test errors
- Writing a broken test before walking away from code can remind you were you left off
- Tests are also code... don't forget to treat them as such

### A very quick mention about test driven design (TDD)...

Idea:
- Add a test
- Run test and verify new test that doesn't pass
- Write code to make new test pass
- Refactor
- Confirm test still passes
- Repeat until code is complete 


Benefits:
- Requires coder to think about making code more modularized, flexible, and independent
- Ideally this acts as a forcing function to think in focused testable units... not huge blobs of code, but building blocks that are easily tested separately and later combined together to achieve the desired coding objective 

#### "Code Smells"
- If you have a hard time testing a single function you might need to split it up

### Unittest
- https://docs.python.org/3.8/library/unittest.html
- Python builtin testing library

- Lots of asserts to remember...

- Can call tests in a multitude of ways

In [ ]:
python -m unittest -h
python -m unittest test_module1 test_module2
python -m unittest test_module.TestClass
python -m unittest test_module.TestClass.test_method
python -m unittest -v test_module
python -m unittest discover <test_directory>
python -m unittest discover -s <directory> -p '*_test.py'

- Various ways to skip tests

In [ ]:
@unittest.skip("demonstrating skipping")
@unittest.skipIf(sys.version_info[0] > 2, "Python 2 only test")
@unittest.skipUnless(sys.platform.startswith("win"), "Only for Windows")

- If setUp() succeeded, the tearDown() method will be run whether runTest() succeeded or not.


### Pytest
- https://docs.pytest.org/en/stable/contents.html
- pytest --help
- From unittest docs... "pytest: Third-party unittest framework with a lighter-weight syntax for writing tests. For example, assert func(10) == 42"
- Tests can be functions or classes similar to unittest
- Supports unittest and other runners
- Much more options, plugins, etc compared to unittest
- Seems to be industry standard
- Assert has special powers in pytest, no need to remember all the unittest asserts

#### Parametrize testing 

In [ ]:
@pytest.mark.parametrize('nom, denom, result', 
                         [(2, 4, .5), 
                          (1, 3, pytest.approx(.333, .1))])
def test_division_with_parametrize(nom, denom, result):
    """Same as division with two ints, but with parametrize"""
    assert division(nom, denom) == result

#### Fixtures 

- Fixtures are similar to setup and teardown in unittest 
- Some args like autouse means these fixtures will be auto invoked at their respective scopes
- Args can also include fixture scope: function, class, module, or session

- conftest.py is module pytest knows to look for scope specific fixtures.. all sub folders have access to conftest, can have multiple. Doesn't need to be imported

In [ ]:
import pytest

In [ ]:
%%bash
pytest --fixtures
# builtin fixtures available for usage... 

- ^ builtin fixtures available for usage... 
- Items such as tmpdir fixtures allow you to create temporary directories to create files or directories 

- Can mark tests using a decorator... will complain though if you don't add to ini file 

In [ ]:
@pytest.mark.this
@pytest.mark.that
pytest -m "this and that"
pytest -m "this or that"

- Can skip tests like unittest as well...

In [ ]:
@pytest.mark.skip(reason='misunderstood the API')
@pytest.mark.skipif(sys.version_info[0] > 2, reason="Python 2 only test")
@pytest.mark.xfail(sys.version_info[0] > 2, reason="Python 2 only test") # mark expected to fail

## Tools for unit testing

### Hypothesis 
- https://hypothesis.readthedocs.io/en/latest/details.html
- Show stats with --hypothesis-show-statistics

In [ ]:
@given(st.integers(), st.integers())
def test_with_hypothesis_ints(num, denom):
    """Stress test for potential errors"""
    #assume(denom > 0)
    division(num, denom)

### Stubs and Mocks
- https://docs.python.org/3/library/unittest.mock.html (since 3.3)
- Stubs for overwriting a namespace and providing a return value
- Mocks for overwriting a namespace and checking behavior (did it get called, etc...)
- Really good: https://www.youtube.com/watch?v=ww1UsGZV8fQ
- pytest also has monkey patching... we'll look at unittest mock today though

#####  When to consider a stub or mock: 
- To bypass complicated code 
- Remove outside dependencies from a test
- Avoid using something that is overly slow (Tests should be fast)
- Represent external resources (Avoid networks, databases, etc)
- Supply non-predictable values
- The code in question doesn't exist... test while waiting for others to write their code

###### "Code Smells"
- Red flags should be raised when everything is patched... potentially means a lot of dependencies the way code is written


### Coverage
- How much of your code did you touch with testing?
- Doesn't necessary mean it was good testing, just that it was exercised

In [ ]:
coverage run -m pytest src/examples/coding.py src/testing/test_coding_with_pytest.py 
coverage run -m pytest
coverage report
coverage html

- Let's add some short circuit code in the original coding.py...

In [ ]:
if False and this_wont_raise_an_error(): 
      but_linting_will_find_it() 

### Tox
- https://tox.readthedocs.io/en/latest/
- Test code in multiple versions of python using a virtual env
- Good for more than just py2 to py3 checks, but also different py3 versions

In [ ]:
% cat tox.ini 
[tox]
envlist = py38,py27
skipsdist = True # get around no setup.py

[testenv]
deps = pytest
commands = python -m pytest src/testing/ -v --doctest-modules
 